## Install Dependencies

In [1]:
# %pip install earthengine-api -q
# %pip install git+https://github.com/cordmaur/GEES2Downloader.git@main -q
# %pip install geemap -q

In [1]:
# %pip install pympler

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
import ee
import matplotlib.pyplot as plt
from pathlib import Path

import xarray as xr
import rasterio as rio

from waterdetect.engine import search_tiles, WaterDetect
from waterdetect.cloudless import get_gee_img

import os
os.environ['PROJ_LIB'] = r'D:\Programs\miniconda3\envs\planetary\Library\share\proj'
os.environ['GDAL_DATA'] = r'D:\Programs\miniconda3\envs\planetary\Library\share'


In [4]:
# ee.Authenticate()
ee.Initialize()

In [18]:
# For memory tracking

# %pip install pympler

from IPython.display import clear_output

# from pympler.tracker import SummaryTracker
from pympler.classtracker import ClassTracker
# tracker = SummaryTracker()
tr = ClassTracker()
tr.track_class(WaterDetect)

# tracker.print_diff()

# Getting the Images to be Processed

In [19]:
tiles = search_tiles('22KFV', '2019-10')
tiles

[<Item id=S2B_MSIL2A_20191030T133229_R081_T22KFV_20201004T074719>,
 <Item id=S2B_MSIL2A_20191027T132239_R038_T22KFV_20201004T065003>,
 <Item id=S2A_MSIL2A_20191025T133231_R081_T22KFV_20201004T060431>,
 <Item id=S2A_MSIL2A_20191022T132241_R038_T22KFV_20201004T045948>,
 <Item id=S2B_MSIL2A_20191020T133229_R081_T22KFV_20201004T042823>,
 <Item id=S2B_MSIL2A_20191017T132239_R038_T22KFV_20201004T032642>,
 <Item id=S2A_MSIL2A_20191015T133231_R081_T22KFV_20201004T023230>,
 <Item id=S2A_MSIL2A_20191012T132241_R038_T22KFV_20201004T012531>,
 <Item id=S2B_MSIL2A_20191010T133229_R081_T22KFV_20201004T005150>,
 <Item id=S2B_MSIL2A_20191007T132239_R038_T22KFV_20201003T234537>,
 <Item id=S2A_MSIL2A_20191005T133231_R081_T22KFV_20201003T224936>,
 <Item id=S2A_MSIL2A_20191002T132241_R038_T22KFV_20201003T214143>]

## Process the Images

In [20]:
import gc
from datetime import datetime

cluster_bands = ['mndwi', 'ndwi', 'B12']

lap = datetime.now()

for tile in tiles:
    clear_output(wait=True)
    tr.create_snapshot()
    tr.stats.print_summary()
    
    
    print('*'*100)
    print(f'Processing tile {tile}')
    wd = WaterDetect(tile, cluster_bands, s2clouds=True, n_jobs=7)
    wd.run_detect_water()
    
    # save thumbnails
    wd.plot_thumbs(cols=3, thumbs=['rgb', 'watermask', 'mndwi', 'mask', 'glint', 'ndwi'],  save_folder='../tmp2')
    
    # save graphs
    wd.plot_graphs([['ndwi', 'mndwi'], ['B12', 'mndwi']], cols=2, save_folder='../tmp2')
    
    # save the watermask
    wd.save_geotiff('nodata_watermask', Path('./tmp')/f'{wd.img_item.id[:38]}_watermask.tif')
        
    print(datetime.now() - lap)
    lap = datetime.now()
    # del wd
    # gc.collect()

---- SUMMARY ------------------------------------------------------------------
                                         active      0     B      average   pct
-------------------------------------------------------------------------------
****************************************************************************************************
Processing tile <Item id=S2B_MSIL2A_20191030T133229_R081_T22KFV_20201004T074719>
Getting image: S2B_MSIL2A_20191030T133229_R081_T22KFV_20201004T074719
Retrieving band info
Dividing band in 4 tiles


Tiles:   0%|          | 0/4 [00:00<?, ?tile/s]

Tile[0:2896,2896:5490]:   0%|          | 0.00/9.77M [00:00<?, ?b/s]

Tile[0:2896,0:2896]:   0%|          | 0.00/10.9M [00:00<?, ?b/s]

Tile[2896:5490,2896:5490]:   0%|          | 0.00/8.75M [00:00<?, ?b/s]

...

D:\OneDrive - Agência Nacional de Águas\Projects\pc_waterdetect\waterdetect\glint.py:54: RuntimeWarning: Mean of empty slice
  return np.nanmean(arrays_lst, axis=0)


resampling B11 to (10980, 10980)
resampling B12 to (10980, 10980)
resampling SCL to (10980, 10980)
Calculating indices
k=2 - Calinski=4663.271931714692
k=3 - Calinski=2910.6538196626025
k=4 - Calinski=1943.2079794682286
k=5 - Calinski=1478.0775338082672
k=6 - Calinski=4484.1924167229245
k=7 - Calinski=3744.0343057945947
Final clustering with k=2
Water cluster = 1
Generalized for the whole scene


ValueError: Unsupported dtype

ValueError: Unsupported dtype

<Figure size 1500x1000 with 6 Axes>

In [24]:
wd.plot_thumbs()

[autoreload of waterdetect.engine failed: Traceback (most recent call last):
  File "D:\Programs\miniconda3\envs\planetary\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "D:\Programs\miniconda3\envs\planetary\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "D:\Programs\miniconda3\envs\planetary\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "D:\Programs\miniconda3\envs\planetary\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "D:\Programs\miniconda3\envs\planetary\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "D:\Programs\miniconda3\envs\planetary\lib\site-packages\IPython\extensions\autoreload.py", line 266, in update_function
    seta

ValueError: Unsupported dtype

<Figure size 1500x1500 with 4 Axes>

In [22]:
tr.create_snapshot()
tr.stats.print_summary()

---- SUMMARY ------------------------------------------------------------------
                                         active      0     B      average   pct


KeyError: 'waterdetect.engine.WaterDetect'